The goal is to provide better data about the top 50 solar flares recorded so far than those shown by SpaceWeatherLive.com. Use this messy NASA data to add more features for the top 50 solar flares. You need to scrape this information directly from each HTML page. You can read here more about Solar Flares, coronal mass ejections, and the solar flare alphabet soup.

https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html (SpaceWeatherLive.com)

https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html (NASA Live)

**Question 1** 
Scrape your competitor's data (10 pts)
Scrape data for the top 50 solar flares shown in SpaceWeatherLive.com

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime

In [2]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",headers=header)
#print(page.status_code)

soup = BeautifulSoup(page.text, 'html.parser')

tab = soup.find('table', class_ = 'table table-striped')

#print(tab)

rank=[]
flare=[]
dat=[]
region=[]
st=[]
max=[]
end=[]
movie=[]
archive=[]

for i in tab.find_all('tbody'):		#to derive data from <tbody> to </tbody>
    rows = i.find_all('tr')
    for row in rows:
        rank.append(row.find_all('td')[0].text)
        fl = row.find('span', class_ = 'flare_small Xclass').text
        flare.append(fl.replace('+', ''))
        dat.append(row.find_all('td')[2].text)
        region.append(row.find('a').text)
        st.append(row.find_all('td')[4].text)
        max.append(row.find_all('td')[5].text)
        end.append(row.find_all('td')[6].text)
        abc=row.find('a',class_='btn btn-xs btn-outline-dark').text
        if abc == 'View archive':
            movie.append('')
        else:
            movie.append('Movie')
        archive.append(row.find_next('a', class_='btn btn-xs btn-outline-dark',string=re.compile("View")).text)
        
#to store data into dataframe
df = pd.DataFrame({'Rank':rank,'X_Classification':flare,'Date':dat,'Region':region,'Start Time':st,'Maximum Time':max,'End Time':end,'Movie':movie,'Archive':archive}) 
df1=df.drop(['Movie', 'Archive'], axis=1)  
print(df1) 
#df.to_csv('task1.csv') 


   Rank X_Classification        Date Region Start Time Maximum Time End Time
0     1              X28  2003/11/04   0486      19:29        19:53    20:06
1     2              X20  2001/04/02   9393      21:32        21:51    22:03
2     3            X17.2  2003/10/28   0486      09:51        11:10    11:24
3     4              X17  2005/09/07   0808      17:17        17:40    18:03
4     5            X14.4  2001/04/15   9415      13:19        13:50    13:55
5     6              X10  2003/10/29   0486      20:37        20:49    21:01
6     7             X9.4  1997/11/06   8100      11:49        11:55    12:01
7     8             X9.3  2017/09/06   2673      11:53        12:02    12:10
8     9               X9  2006/12/05   0930      10:18        10:35    10:45
9    10             X8.3  2003/11/02   0486      17:03        17:25    17:39
10   11             X8.2  2017/09/10   2673      15:35        16:06    16:31
11   12             X7.1  2005/01/20   0720      06:36        07:01    07:26

**Question 2**
Tidy the top 50 solar flare data (10 pts)
Make sure this table is usable using pandas:
1.	Drop the last column of the table, since we are not going to use it moving forward.
2.	Use datetime import to combine the date and each of the three time columns into three datetime columns. You will see why this is useful later on. iterrows() should prove useful here.
3.	Update the values in the dataframe as you do this. Set_value should prove useful.
4.	Set regions coded as - as missing (NaN). You can use dataframe.replace() here.


In [3]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",headers=header)
#print(page.status_code)

soup = BeautifulSoup(page.text, 'html.parser')

tab = soup.find('table', class_ = 'table table-striped')

rank=[]
flare=[]
dat=[]
region=[]
st=[]
max=[]
end=[]
movie=[]
archive=[]

for i in tab.find_all('tbody'):
    rows = i.find_all('tr')
    for row in rows:
        rank.append(row.find_all('td')[0].text)
        fl = row.find('span', class_ = 'flare_small Xclass').text
        flare.append(fl.replace('+', ''))
        dat.append(row.find_all('td')[2].text)
        region.append(row.find('a').text)    
        st.append(row.find_all('td')[4].text)
        max.append(row.find_all('td')[5].text)
        end.append(row.find_all('td')[6].text)
        abc=row.find('a',class_='btn btn-xs btn-outline-dark').text
        if abc == 'View archive':
            movie.append('')
        else:
            movie.append('Movie')
        archive.append(row.find_next('a', class_='btn btn-xs btn-outline-dark',string=re.compile("View")).text)
         

df1 = pd.DataFrame({'Rank':rank,'X_Classification':flare,'Date':dat,'Region':region,'Start Time':st,'Maximum Time':max,'End Time':end,'Movie':movie,'Archive':archive}) 
df=df1.drop(['Movie', 'Archive'], axis=1)  # 1 Drop column
st_dt = pd.to_datetime(df['Date'] + ' ' + df['Start Time']) #2 use datetime
max_dt = pd.to_datetime(df['Date'] + ' ' + df['Maximum Time']) #2 use datetime
end_dt = pd.to_datetime(df['Date'] + ' ' + df['End Time'])  #2 use datetime
df = pd.DataFrame({'Rank':rank,'X_Classification':flare,'Start DateTime':st_dt,'Maximum DateTime':max_dt,'End DateTime':end_dt, 'Region':region})  #3 update value
df = df.replace('-','NaN')      #4 replace '-' with 'NAN' 
print(df)
#df.to_csv('task2.csv') 


   Rank X_Classification      Start DateTime    Maximum DateTime  \
0     1              X28 2003-11-04 19:29:00 2003-11-04 19:53:00   
1     2              X20 2001-04-02 21:32:00 2001-04-02 21:51:00   
2     3            X17.2 2003-10-28 09:51:00 2003-10-28 11:10:00   
3     4              X17 2005-09-07 17:17:00 2005-09-07 17:40:00   
4     5            X14.4 2001-04-15 13:19:00 2001-04-15 13:50:00   
5     6              X10 2003-10-29 20:37:00 2003-10-29 20:49:00   
6     7             X9.4 1997-11-06 11:49:00 1997-11-06 11:55:00   
7     8             X9.3 2017-09-06 11:53:00 2017-09-06 12:02:00   
8     9               X9 2006-12-05 10:18:00 2006-12-05 10:35:00   
9    10             X8.3 2003-11-02 17:03:00 2003-11-02 17:25:00   
10   11             X8.2 2017-09-10 15:35:00 2017-09-10 16:06:00   
11   12             X7.1 2005-01-20 06:36:00 2005-01-20 07:01:00   
12   13             X6.9 2011-08-09 07:48:00 2011-08-09 08:05:00   
13   14             X6.5 2006-12-06 18:29:00 200

**Question 3**
Scrape the NASA data (15 pts)
Next, you need to scrape NASA data to get additional features about these solar flares. This table format is described here.
Once scraped, do the next steps:
1.	Use BeautifulSoup functions (e.g., find, findAll) and string functions (e.g., split and built-in slicing capabilities) to obtain each row of data as a long string.
2.	Use the split function to separate each line of text into a data row.
3.	Create a DataFrame with the data from the table.
4.	Choose appropriate names for columns.


In [4]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",headers=header)
print(page.status_code)

soup = BeautifulSoup(page.content, 'html.parser')
tab = soup.find('pre')
dat = tab.get_text()
rows = dat.split('\n') #split into data into row

for i in range(0,12):
    rows.pop(0);    #Popping First Twelve Lines of <pre>
rows.pop(len(rows)-1)   #Pop Last two Line before </pre>
rows.pop(len(rows)-1)
df = pd.DataFrame(columns=['Start Date', 'Start Time', 'End Date', 'End Time', 'Start Frequency', 'End Frequency', 'Flare Location', 'Flare Region','Flare Classification', 'CME Date', 'CME Time', 'CME CPA', 'CME Width', 'CME Speed','Plot'], index = range(0,len(rows)))

i = 0   #count row of table
for row in rows:
    column = row.split(' ')
    while '' in column:
        column.remove('')
    j = 0   #count column of table
    while j < 15:
        df.iat[i, j] = column[j]
        j += 1
    i += 1    
print(df) 
#df.to_csv('task3.csv')

200
     Start Date Start Time End Date End Time Start Frequency End Frequency  \
0    1997/04/01      14:00    04/01    14:15            8000          4000   
1    1997/04/07      14:30    04/07    17:30           11000          1000   
2    1997/05/12      05:15    05/14    16:00           12000            80   
3    1997/05/21      20:20    05/21    22:00            5000           500   
4    1997/09/23      21:53    09/23    22:16            6000          2000   
..          ...        ...      ...      ...             ...           ...   
517  2017/09/17      11:45    09/17    12:35           16000           900   
518  2017/10/18      05:48    10/18    12:40           16000           400   
519  2019/05/03      23:52    05/04    00:16           13000          2300   
520  2020/11/29      13:07    11/29    15:23           14000           850   
521  2020/12/07      16:18    12/08    02:00           14000           160   

    Flare Location Flare Region Flare Classification CME Da

**Question 4**
Tidy the NASA table (15 pts)
Here we will code missing observations properly, recode columns that correspond to more than one piece of information, and treat dates and times appropriately.
1.	Recode any missing entries as NaN. Refer to the data description to see how missing entries are encoded in each column. Be sure to look carefully at the actual data, as the nasa descriptions might not be completely accurate.
2.	The CPA column (cme_angle) contains angles in degrees for most rows, except for halo flares, which are coded as Halo. Create a new column that indicates if a row corresponds to a halo flare or not, and then replace Halo entries in the cme_angle column with NaN.
3.	The width column indicates if the given value is a lower bound. Create a new column that indicates if width is given as a lower bound, and remove any non-numeric part of the width column.
4.	Combine date and time columns for start, end and cme so they can be encoded as datetime objects.


In [5]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",headers=header)
print(page.status_code)

soup = BeautifulSoup(page.content, 'html.parser')
tab = soup.find('pre')
dat = tab.get_text()
rows = dat.split('\n') #split into data into row

for i in range(0,12):
    rows.pop(0);    #Popping First Twelve Lines of <pre>
rows.pop(len(rows)-1)   #Pop Last two Line before </pre>
rows.pop(len(rows)-1)
df = pd.DataFrame(columns=['Start Date', 'Start Time', 'End Date', 'End Time', 'Start Frequency', 'End Frequency', 'Flare Location', 'Flare Region','Flare Classification', 'CME Date', 'CME Time', 'CME CPA', 'CME Width', 'CME Speed','Plot'], index = range(0,len(rows)))

i = 0   #count row of table
for row in rows:
    column = row.split(' ')
    while '' in column:
        column.remove('')
    j = 0   #count column of table
    while j < 15:
        df.iat[i, j] = column[j]
        j += 1
    i += 1    
df = df.replace('????','NaN')   #Start and End frequencies replacement
df = df.replace('----','NaN')
df = df.replace('-----','NaN')
df = df.replace('--/--','NaN')
df = df.replace('--:--','NaN')

df.loc[df['CME CPA'] == "Halo", 'is_halo'] = 'True' 
df.loc[df['CME CPA'] != "Halo", 'is_halo'] = 'False'
df = df.replace('Halo','NaN')

df['width_lower_bound'] = df['CME Width'].apply(lambda x: 'False' if x.isdigit() else 'True')
df['CME Width'] = df['CME Width'].str.replace('>',' ')
st_dt = pd.to_datetime(df['Start Date'] + ' ' + df['Start Time']) #2 use datetime
end_dt = pd.to_datetime(df['Start Date'] + ' ' + df['End Time'], errors='ignore') #2 use datetime
cme_dt = pd.to_datetime(df['Start Date'] + ' ' + df['CME Time'],errors='ignore') #2 use datetime

df=df.drop(['Start Date', 'Start Time', 'End Date', 'End Time', 'CME Date', 'CME Time'], axis=1)			#Created Dataframe with names
df.insert(0, "Start DateTime", st_dt)
df.insert(1, "End DateTime", end_dt)
df.insert(7, "CME DateTime", cme_dt)

print(df) 
#df.to_csv('task4.csv')

200
         Start DateTime      End DateTime Start Frequency End Frequency  \
0   1997-04-01 14:00:00  1997/04/01 14:15            8000          4000   
1   1997-04-07 14:30:00  1997/04/07 17:30           11000          1000   
2   1997-05-12 05:15:00  1997/05/12 16:00           12000            80   
3   1997-05-21 20:20:00  1997/05/21 22:00            5000           500   
4   1997-09-23 21:53:00  1997/09/23 22:16            6000          2000   
..                  ...               ...             ...           ...   
517 2017-09-17 11:45:00  2017/09/17 12:35           16000           900   
518 2017-10-18 05:48:00  2017/10/18 12:40           16000           400   
519 2019-05-03 23:52:00  2019/05/03 00:16           13000          2300   
520 2020-11-29 13:07:00  2020/11/29 15:23           14000           850   
521 2020-12-07 16:18:00  2020/12/07 02:00           14000           160   

    Flare Location Flare Region Flare Classification      CME DateTime  \
0           S25E16   

**Question 5**
Can you replicate the top 50 solar flare table in SpaceWeatherLive.com exactly using the data obtained from NASA? That is if you get the top 50 solar flares from the NASA table based on their classification (e.g., X28 is the highest), do you get data for the same solar flare events? Include code used to get the top 50 solar flares from the NASA table (be careful when ordering by classification). Write a sentence or two discussing how well you can replicate the SpaceWeatherLive data from the NASA data.

In [6]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",headers=header)
print(page.status_code)
#print(data.text)
soup = BeautifulSoup(page.content, 'html.parser')
tab = soup.find('pre')
dat = tab.get_text()
rows = dat.split('\n') #split into data into row
for i in range(0,12):
    rows.pop(0);    #Popping First Twelve Lines of <pre>
rows.pop(len(rows)-1)   #Pop Last two Line before </pre>
rows.pop(len(rows)-1)
df = pd.DataFrame(columns=['Start Date', 'Start Time', 'End Date', 'End Time', 'Start Frequency', 'End Frequency', 'Flare Location', 'Flare Region','Flare Classification', 'CME Date', 'CME Time', 'CME CPA', 'CME Width', 'CME Speed','Plot'], index = range(0,len(rows)))
i = 0   #count row of table
for row in rows:
    column = row.split(' ')
    while '' in column:
        column.remove('')
    j = 0   #count column of table
    while j < 15:
        df.iat[i, j] = column[j]
        j += 1
    i += 1    
df = df.replace('????','NaN')   #Start and End frequencies replacement
df = df.replace('----','NaN')
df = df.replace('-----','NaN')
df = df.replace('--/--','NaN')
df = df.replace('--:--','NaN')

df.loc[df['CME CPA'] == "Halo", 'is_halo'] = 'True' 
df.loc[df['CME CPA'] != "Halo", 'is_halo'] = 'False'
df = df.replace('Halo','NaN')

df['width_lower_bound'] = df['CME Width'].apply(lambda x: 'False' if x.isdigit() else 'True')
df['CME Width'] = df['CME Width'].str.replace('>',' ')

df=df.sort_values(by='Flare Classification', inplace=False,ascending=False).head(50)
df['Flare Classification'] = df['Flare Classification'].str.replace('X',' ')
 
df["Flare Classification"]=pd.to_numeric(df["Flare Classification"], errors='ignore')
df=df.sort_values(by='Flare Classification',ascending=False).head(50)
df['Flare Classification'] = 'X' + df['Flare Classification'].astype(str)

print(df)
#df.to_csv('task5.csv')


200
     Start Date Start Time End Date End Time Start Frequency End Frequency  \
240  2003/11/04      20:00    11/04    24:00           10000           200   
117  2001/04/02      22:05    04/03    02:30           14000           250   
233  2003/10/28      11:10    10/29    24:00           14000            40   
126  2001/04/15      14:05    04/16    13:00           14000            40   
234  2003/10/29      20:55    10/29    24:00           11000           500   
8    1997/11/06      12:20    11/07    08:30           14000           100   
514  2017/09/06      12:05    09/07    08:00           16000            70   
328  2006/12/05      10:50    12/05    20:00           14000           250   
237  2003/11/02      17:30    11/03    01:00           12000           250   
515  2017/09/10      16:02    09/11    06:50           16000           150   
288  2005/01/20      07:15    01/20    16:30           14000            25   
359  2011/08/09      08:20    08/09    08:35           16000

**Question 6**
Write a function that finds the best matching row in the NASA data for each of the top 50 solar flares in the SpaceWeatherLive data. Here, you have to decide for yourself how you determine what is the best matching entry in the NASA data for each of the top 50 solar flares. In your submission, include an explanation of how you are defining best matching rows across the two datasets in addition to the code used to find the best matches. Finally, use your function to add a new column to the NASA dataset indicating its rank according to SpaceWeatherLive, if it appears in that dataset.

In [7]:
#Code of Task 1(Space Weather) and Task 3(Nasa Data) are taken and following code is added to it.
#merge Space Weather and Nasa Dataset in one using merge()
result = pd.merge(df, df1,  left_on=[df["Start Date"],df["Flare Classification"]],right_on=[df1["Date"],df1["X_Classification"]], how="left")
result=result.drop(['X_Classification', 'Start Time_y', 'Region', 'Maximum Time', 'End Time_y','Date'], axis=1)
result
#result.to_csv('task6.csv')


,key_0,key_1,Start Date,Start Time_x,End Date,End Time_x,Start Frequency,End Frequency,Flare Location,Flare Region,...,CME Time,CME CPA,CME Width,CME Speed,Plot,is_halo,width_lower_bound,Rank,Movie,Archive
0,2003/11/04,X28.0,2003/11/04,20:00,11/04,24:00,10000,200,S19W83,10486,...,19:54,NaN,360,2657,PHTX,True,False,NaN,NaN,NaN
1,2001/04/02,X20.0,2001/04/02,22:05,04/03,02:30,14000,250,N19W72,9393,...,22:06,261,244,2505,PHTX,False,False,NaN,NaN,NaN
2,2003/10/28,X17.0,2003/10/28,11:10,10/29,24:00,14000,40,S16E08,10486,...,11:30,NaN,360,2459,PHTX,True,False,NaN,NaN,NaN
3,2001/04/15,X14.0,2001/04/15,14:05,04/16,13:00,14000,40,S20W85,9415,...,14:06,245,167,1199,PHTX,False,False,NaN,NaN,NaN
4,2003/10/29,X10.0,2003/10/29,20:55,10/29,24:00,11000,500,S15W02,10486,...,20:54,NaN,360,2029,PHTX,True,False,NaN,NaN,NaN
5,1997/11/06,X9.4,1997/11/06,12:20,11/07,08:30,14000,100,S18W63,8100,...,12:10,NaN,360,1556,PHTX,True,False,7,Movie,View archive
6,2017/09/06,X9.3,2017/09/06,12:05,09/07,08:00,16000,70,S08W33,12673,...,12:24,NaN,360,1571,PHTX,True,False,8,Movie,View archive
7,2006/12/05,X9.0,2006/12/05,10:50,12/05,20:00,14000,250,S07E68,10930,...,NaN,NaN,NaN,NaN,PHTX,False,True,NaN,NaN,NaN
8,2003/11/02,X8.3,2003/11/02,17:30,11/03,01:00,12000,250,S14W56,10486,...,17:30,NaN,360,2598,PHTX,True,False,10,Movie,View archive
9,2017/09/10,X8.3,2017/09/10,16:02,09/11,06:50,16000,150,S09W92,NaN,...,16:00,NaN,360,3163,PHTX,True,False,NaN,NaN,NaN
